In [1]:
import pandas as pd
import numpy as np

In [2]:
users_final = pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/refs/heads/main/two-tower/users_final_numeric.csv')
products_final= pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/refs/heads/main/two-tower/products_final_numeric.csv')

## Building baseline 2 tower model

### The first issue is that for each training batch, we need to have the same amount of user-item pairs as input. This means we need to use some sort of sampling for each batch in order to make sure they're both the same size.

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate, Dot
from tensorflow.keras.models import Model

In [11]:
user_input = Input(shape=(16,), name='user_input')
item_input = Input(shape=(31,), name='item_input')

In [12]:
#Changed from baseline
user_tower = Dense(256, activation='relu')(user_input)

In [13]:
item_tower = Dense(256, activation='relu')(item_input)

In [14]:
dot_product = Dot(axes=1)([user_tower, item_tower])

In [15]:
model = Model(inputs=[user_input, item_input], outputs=dot_product)

In [16]:
model.compile(optimizer='adam', loss='mse')

In [17]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None, 16)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_input          │ (None, 31)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 256)       │      4,352 │ user_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 256)       │      8,192 │ item_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_1 (Dot)         │ (None, 1)         │          0 │ dense_2[0][0],    │
│                     │                   │            │ dense_3[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 12,544 (49.00 KB)

 Trainable params: 12,544 (49.00 KB)

 Non-trainable params: 0 (0.00 B)

### Formatting inputs

In [18]:
    def create_labels_and_train(users_df, products_df, model, batch_size, num_epochs):
        for epoch in range(num_epochs):
            # generate random user-item pairs through random indices for each batch
            user_indices = np.random.randint(0, len(users_df), size=batch_size)
            product_indices = np.random.randint(0, len(products_df), size=batch_size)

            # extract the data
            user_data = users_df.iloc[user_indices]#.copy()  # copy to avoid SettingWithCopyWarning
            product_data = products_df.iloc[product_indices]#.copy()

            # we will be creating target similarity labels
            target_similarity = []

            # loop through user and product indices to create labels
            for user_idx, product_idx in zip(user_indices, product_indices):
                user_product_id = users_df.iloc[user_idx]['product_id']
                item_product_id = products_df.iloc[product_idx]['product_id']

                # if the user and item product id match, it means the user purchased the product
                # otherwise, there is no interaction and the target similarity would be 0
                target_similarity.append(1 if user_product_id == item_product_id else 0)

            # convert to a numpy array
            target_similarity = np.array(target_similarity)

            # drop 'product_id' from both dataframes
            user_data = user_data.drop(columns=['product_id'])
            product_data = product_data.drop(columns=['product_id', 'name_embedding'])


            # train the model with the pairs
            model.fit([user_data.values, product_data.values], target_similarity, epochs=1, batch_size=batch_size)

    # parameters
    batch_size = 500
    num_epochs = 25

    create_labels_and_train(users_final, products_final, model, batch_size, num_epochs)


c:\Users\asmi_\Asmi's Documents\Academics\2022-26_UCLA\Projects\amex\.venv\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['user_input', 'item_input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 470ms/step - loss: 557676069126144.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 489949468033024.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 488882504204288.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 498928701145088.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 407302553206784.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 411474879053824.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 391743665078272.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 384306794987520.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 358104004820992.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 335603979780096.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 306749986832384.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 333642219913216.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 254587005042688.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 286118272565248.0000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 245483972853760